# 1. Import

In [2]:
!pip install transformers requests beautifulsoup4

     ---------------------------------------- 4.4/4.4 MB 15.8 MB/s eta 0:00:00
     -------------------------------------- 101.5/101.5 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 42.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model BERT

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

# 3. Encode and calculate sentiment

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [12]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [16]:
torch.argmax(result.logits)

tensor(3)

In [15]:
score = int(torch.argmax(result.logits)) + 1
print("the sentiment score of the sentence from (/5) is ", score)

the sentiment score of the sentence from (/5) is  4


In [17]:
tokens2 = tokenizer.encode('I hate it you are the worst.', return_tensors='pt')
result2 = model(tokens2)
score2 = int(torch.argmax(result2.logits)) + 1
print("the sentiment score of the sentence from (/5) is ", score2)

the sentiment score of the sentence from (/5) is  1


# 4. Collect Review

In [18]:
#web scrapping
r = requests.get('https://www.yelp.com/biz/pearls-deluxe-burgers-san-francisco-3')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [24]:
reviews

["Help! Me and my wife've been taking burgers from here almost daily and still can't find a stop to it!!! Sooo satisfyingly delicious!!! Juicy meat, fluffy bun with a nice crispy shell, amazing sauces. Cant believe myself walking passing by this place to get an asian groceries next door all the time never realize that there's such an awesome burger place right under my nose!Everything we got from here was so good! Blue cheese, mushroom, avocado, bula burger, pesto, thinking of trying everything ion the menu because I have confidence that they won't disappoint me. Hands down best burger in town.The fries are so fresh and fluffy. Garlic fries will absolutely give you a stinky breath but I just can't stop eating itMilkshake is kinda too sweet for me but it's an acceptable sweetness that I still can enjoy, love the various options for the shake's flavors, sadly they don't serve the seasonal cheesecake milkshake anymore. Also, lovely service is something worth mentioning!",
 "We recently gr

# 5. Load Reviews into Dataframe and Score

In [25]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [26]:
df

,review
0,Help! Me and my wife've been taking burgers fr...
1,"We recently grabbed dinner here, and although ..."
2,One of the few burger spots in SF! Being a bur...
3,Pretty good burger by SF standards! (they are ...
4,When searching for some burger candidates stum...
5,I was visiting SF with a friend and we didn't ...
6,Hood little spot got a quick lunch friendly st...
7,This place doesn't look like much from the out...
8,Pearl's Deluxe Burgers is a San Francisco clas...
9,First time at Pearl's Deluxe Burgers today and...


In [30]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [31]:
df['review'].iloc[0]

"Help! Me and my wife've been taking burgers from here almost daily and still can't find a stop to it!!! Sooo satisfyingly delicious!!! Juicy meat, fluffy bun with a nice crispy shell, amazing sauces. Cant believe myself walking passing by this place to get an asian groceries next door all the time never realize that there's such an awesome burger place right under my nose!Everything we got from here was so good! Blue cheese, mushroom, avocado, bula burger, pesto, thinking of trying everything ion the menu because I have confidence that they won't disappoint me. Hands down best burger in town.The fries are so fresh and fluffy. Garlic fries will absolutely give you a stinky breath but I just can't stop eating itMilkshake is kinda too sweet for me but it's an acceptable sweetness that I still can enjoy, love the various options for the shake's flavors, sadly they don't serve the seasonal cheesecake milkshake anymore. Also, lovely service is something worth mentioning!"

In [33]:
sentiment_score(df['review'].iloc[0])

5

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df

,review,sentiment
0,Help! Me and my wife've been taking burgers fr...,5
1,"We recently grabbed dinner here, and although ...",2
2,One of the few burger spots in SF! Being a bur...,3
3,Pretty good burger by SF standards! (they are ...,4
4,When searching for some burger candidates stum...,5
5,I was visiting SF with a friend and we didn't ...,5
6,Hood little spot got a quick lunch friendly st...,4
7,This place doesn't look like much from the out...,4
8,Pearl's Deluxe Burgers is a San Francisco clas...,5
9,First time at Pearl's Deluxe Burgers today and...,5


In [36]:
df['review'].iloc[1]

"We recently grabbed dinner here, and although it's somewhat of a SF staple, it really fell short for me. I got the prized pearl, which simply had an overwhelming amount of blue cheese. I do really enjoy blue cheese, but towards the middle of the burger, it felt as if I was just eating bites of blue cheese over and over. The onions were in between raw and grilled (still opaque), so were in a weird state where onion could actually detract from a burger. The patty itself was not well seasoned; just tasted like I was biting into unseasoned cooked beef. By the end, all the juices had flowed out of everything, and I was eating basically slop.Personally will not be coming back!"